In [1]:
%run /home/jburns/RFCU_DataScience/scripts/Jupyter_init_config.py \
    --app_name 'adobe_exploration' \
    --executor_memory '2g' \
    --driver_memory '4g' \
    --executor_cores 1 \
    --max_cores 1 \
    --driver_cores 1 \
    --driver_max_Resultsize '2g' \
    --pd_cols 600 \
    --pd_rows 200

usage: Jupyter_init_config.py [-h] [--app_name APP_NAME]
                              [--executor_memory EXECUTOR_MEMORY]
                              [--executor_cores EXECUTOR_CORES]
                              [--max_cores MAX_CORES]
                              [--driver_cores DRIVER_CORES]
                              [--driver_memory DRIVER_MEMORY]
                              [--driver_max_Resultsize DRIVER_MAX_RESULTSIZE]
                              [--local_spark LOCAL_SPARK] [--pd_cols PD_COLS]
                              [--pd_rows PD_ROWS]

optional arguments:
  -h, --help            show this help message and exit
  --app_name APP_NAME, -app_name APP_NAME
                        REQUIRED: App Name
  --executor_memory EXECUTOR_MEMORY, -executor_memory EXECUTOR_MEMORY
                        Executor Memory
  --executor_cores EXECUTOR_CORES, -executor_cores EXECUTOR_CORES
                        Executor Cores
  --max_cores MAX_CORES, -max_cores MAX_CORES
        

## Start by importing all days into a single mdf

In [2]:
%%time

import pandas as pd
import glob
# "../data_folder/data.csv"
# Here we will read in the data, and combine it with the column names .tsv
path = '..Adobe_lookup_data/'
all_files = glob.glob("*.tsv")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, sep='\t', header = None)
    li.append(df)

mdf = pd.concat(li, axis=0, ignore_index=True)
headers = pd.read_csv('Adobe_lookup_data/column_headers.tsv',sep='\t')
mdf.columns = headers.columns
#print(mdf.info(verbose = True))

CPU times: user 3.23 s, sys: 311 ms, total: 3.54 s
Wall time: 3.54 s


# Load the dimensional tables and merge to get labels instead of IDs, drop te ID field. -> new MDF 

In [4]:
%%time

# Here we will read in the data, and combine it with the column names .tsv
path = r'Adobe_lookup_data' # use your path
all_files = glob.glob(path + "/*.tsv")

li2 = []
db = {}

# FOR CONNOR: So here I tried to name all of them dynamically, but the problem is I can't use the variable name 
# "filename" on the left side when trying to name each dataframe individually. How would I do this in the even I had to read in
# like 500 dimensional tables?

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, sep='\t', header = None, encoding = 'ISO-8859-1')
    filename = filename.replace('.tsv', '').replace('Adobe_lookup_data/', '')
    #print(filename) # to print a list of the dictionary keys to screen for hand typing
    # Here almost all dim tables are only 2 columns, except for one. However best information is in second column. Third column is same info just writte worse.
    df = df.iloc[:,:2] 
    df.columns = ['key', 'value']
    db[filename] = df
event = db['event']
js_version = db['javascript_version']
resolution = db['resolution']
referrer_type = db['referrer_type']
lang = db['languages']
plugins = db['plugins']
color = db['color_depth']
connection = db['connection_type']
search_engines = db['search_engines']
country = db['country']
os = db['operating_systems']
browser = db['browser']
browser_type = db['browser_type']

# Now we need to replace all the keys with the value from the dimensional table

# {'browser':'browser',
# 'browser':'browser_type'}
# for key, values in dict_1:
#     df = read_csv()
#     if len(values)>0:
#         df.columns = ['key'] + [x for x in values]
#     else:
#         df.columns = ['key',value]
#     mdf = mdf.merge(df, left_on=value)

# drop_list = []
# df = df.drop(deop_lis,axis=1) # loop through adding all keys, append to drop list, drop list from mdf at end

# For event, I was unable to find any column that uses the keys in the dimension table. I will wait for EDA to see if unique vals on any columns in 
# mdf come back with key vals
#mdf.merge(event, left_on='IDK', right_on='key').drop(['IDK','key'], axis=1).rename(columns={'value':'event'})
mdf = mdf.merge(js_version, left_on='javascript', right_on='key').drop(['javascript','key'], axis=1).rename(columns={'value':'js_version'})
mdf = mdf.merge(resolution, left_on='resolution', right_on='key').drop(['resolution','key'], axis=1).rename(columns={'value':'resolution'})
mdf = mdf.merge(referrer_type, left_on='ref_type', right_on='key').drop(['ref_type','key'], axis=1).rename(columns={'value':'refferer_type'})
mdf = mdf.merge(lang, left_on='language', right_on='key').drop(['language','key'], axis=1).rename(columns={'value':'language'})
# Same with plugin and color as event.
#mdf = mdf.merge(plugins, left_on='IDK', right_on='key').drop(['IDK','key'], axis=1).rename(columns={'value':'plugins'})
#mdf = mdf.merge(color, left_on='language', right_on='key').drop(['language','key'], axis=1).rename(columns={'value':'language'})
mdf = mdf.merge(connection, left_on='connection_type', right_on='key').drop(['connection_type','key'], axis=1).rename(columns={'value':'connection_type'})
mdf = mdf.merge(search_engines, left_on='post_search_engine', right_on='key').drop(['post_search_engine','key'], axis=1).rename(columns={'value':'post_search_engine'})
mdf = mdf.merge(search_engines, left_on='visit_search_engine', right_on='key').drop(['visit_search_engine','key'], axis=1).rename(columns={'value':'visit_search_engine'})
mdf = mdf.merge(country, left_on='country', right_on='key').drop(['country','key'], axis=1).rename(columns={'value':'country'}) 
mdf = mdf.merge(os, left_on='os', right_on='key').drop(['os','key'], axis=1).rename(columns={'value':'os'})
mdf = mdf.merge(browser, left_on='browser', right_on='key').drop('key', axis=1).rename(columns={'value':'browser_name'})
mdf = mdf.merge(browser_type, left_on='browser', right_on='key').drop(['browser','key'], axis=1).rename(columns={'value':'browser_type'})
mdf

,accept_language,carrier,date_time,domain,exclude_hit,first_hit_page_url,first_hit_pagename,first_hit_referrer,first_hit_time_gmt,geo_city,geo_country,geo_dma,geo_region,geo_zip,ip,last_hit_time_gmt,last_purchase_num,last_purchase_time_gmt,mcvisid,mobile_id,new_visit,post_browser_height,post_browser_width,post_campaign,post_channel,post_cookies,post_currency,post_cust_hit_time_gmt,post_evar1,post_evar2,post_evar3,post_evar4,post_evar5,post_evar6,post_evar7,post_evar8,post_evar9,post_evar10,post_evar11,post_evar12,post_evar13,post_evar14,post_evar15,post_evar16,post_evar17,post_evar18,post_evar19,post_evar20,post_evar21,post_evar22,post_evar23,post_evar24,post_evar25,post_evar26,post_evar27,post_evar28,post_evar29,post_evar30,post_evar31,post_evar32,post_evar33,post_evar34,post_evar35,post_evar36,post_evar37,post_evar38,post_evar39,post_evar40,post_evar41,post_evar42,post_evar43,post_evar44,post_evar45,post_evar46,post_evar47,post_evar48,post_evar49,post_evar50,post_evar51,post_evar52,post_evar53,post_evar54,post_evar55,post_evar56,post_evar57,post_evar58,post_evar59,post_evar60,post_evar61,post_evar62,post_evar63,post_evar64,post_evar65,post_evar66,post_evar67,post_evar68,post_evar69,post_evar70,post_evar71,post_evar72,post_evar73,post_evar74,post_evar75,post_evar76,post_evar77,post_evar78,post_evar79,post_evar80,post_evar81,post_evar82,post_evar83,post_evar84,post_evar85,post_evar86,post_evar87,post_evar88,post_evar89,post_evar90,post_evar91,post_evar92,post_evar93,post_evar94,post_evar95,post_evar96,post_evar97,post_evar98,post_evar99,post_evar100,post_event_list,post_hier1,post_hier2,post_hier3,post_hier4,post_hier5,post_keywords,post_page_event,post_page_event_var1,post_page_event_var2,post_page_event_var3,post_page_type,post_page_url,post_pagename,post_pagename_no_url,post_persistent_cookie,post_product_list,post_prop1,post_prop2,post_prop3,post_prop4,post_prop5,post_prop6,post_prop7,post_prop8,post_prop9,post_prop10,post_prop11,post_prop12,post_prop13,post_prop14,post_prop15,post_prop16,post_prop17,post_prop18,post_prop19,post_prop20,post_prop21,post_prop22,post_prop23,post_prop24,post_prop25,post_prop26,post_prop27,post_prop28,post_prop29,post_prop30,post_prop31,post_prop32,post_prop33,post_prop34,post_prop35,post_prop36,post_prop37,post_prop38,post_prop39,post_prop40,post_prop41,post_prop42,post_prop43,post_prop44,post_prop45,post_prop46,post_prop47,post_prop48,post_prop49,post_prop50,post_prop51,post_prop52,post_prop53,post_prop54,post_prop55,post_prop56,post_prop57,post_prop58,post_prop59,post_prop60,post_prop61,post_prop62,post_prop63,post_prop64,post_prop65,post_prop66,post_prop67,post_prop68,post_prop69,post_prop70,post_prop71,post_prop72,post_prop73,post_prop74,post_prop75,post_purchaseid,post_referrer,post_state,post_visid_high,post_visid_low,post_zip,prev_page,ref_domain,service,user_agent,visit_keywords,visit_num,visit_page_num,visit_referrer,visit_start_page_url,visit_start_pagename,visit_start_time_gmt,js_version,resolution,refferer_type,language,connection_type,post_search_engine,visit_search_engine,country,os,browser_name,browser_type
0,"en-US,en;q=0.9",NaN,2021-12-16 00:42:02,comcast.net,0,https://www.redfcu.org/,redfcu|homepage,https://www.bing.com/,1615919718,huntsville,usa,691,al,35806,68.34.130.94,1639607418,0,0,03625729265609618694173234958847762850,0,1,913,1280,NaN,NaN,Y,USD,1639636922,https://www.redfcu.org/,NaN,redfcu|homepage,NaN,NaN,NaN,NaN,NaN,Repeat,Less than 1 day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03625729265609618694173234958847762850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"100,102,108,109,122",NaN,NaN,NaN,NaN,NaN,::empty::,0,NaN,NaN,NaN,NaN,https://www.redfcu.org,redfcu|homepage,redfcu|homepage,Y,;;;;,https://www

# Give me a snapshot of where the nulls are:

1. group by column
2. use pandas.groupby.agg() function to get count of rows, count of nulls, rate of nulls
3. generate a list of 100% columns -> null_col_list (if you insist, save list as a 2022_02_11_null_cols.txt file)
4. create MDF_non_null dataframe where you've dropped the non_null_cols

In [5]:
# Checking for columns with only null values in them
null_cols_list = []
for column in mdf:
    all_null = pd.isnull(mdf[column]).all()
    if all_null == True:
        null_cols_list.append(column)

# Drop the null columns from the dataframe.
mdf_non_null = mdf.drop(labels = null_cols_list, axis = 1)

# Analyze the columns with some but not all null vals.
col_space = mdf_non_null.columns
null_info_df = pd.DataFrame(index=col_space)
for column in col_space:
    null_count = mdf[column].isnull().sum()
    null_rate = mdf[column].isnull().mean()
    row_count = mdf[column].size
    null_info_df.at[column,'null_count'] = null_count
    null_info_df.at[column,'row_count'] = row_count
    null_info_df.at[column, 'null_rate'] = round(null_rate * 100,2)
    
null_info_df = null_info_df.sort_values(by=['null_rate'], axis=0, ascending=False)
null_info_df

,null_count,row_count,null_rate
post_evar22,16785.0,16786.0,99.99
post_evar20,16785.0,16786.0,99.99
post_page_event_var2,16783.0,16786.0,99.98
post_evar4,16755.0,16786.0,99.82
post_evar5,16755.0,16786.0,99.82
post_prop27,14960.0,16786.0,89.12
post_prop26,14319.0,16786.0,85.30
carrier,12750.0,16786.0,75.96
post_page_event_var1,9005.0,16786.0,53.65
visit_start_pagename,7787.0,16786.0,46.39


# EDA:

ideas:
- group by values, count(*) order by cnt - display head(10)
    - common values
    same value?
    distribution?
- null rate in that column


In [70]:
# Columnwise EDA function:

def column_eda(series_name):
    series = mdf_non_null[series_name]
    null_rate = round(series.isnull().mean() * 100, 2)
    grouped = series.groupby(series).agg('count').sort_values(ascending = False).head(10)
    str = f"Null rate: {null_rate}\n"
    print(str)
    print(grouped)

# Accept language

- Accept language unsure how it differs from lanuage. 
- Could possibly need to be truncated to first 5 characters if the additional information does not provide value to the data.
- Review after EDA to determine if we want to keep this, lanugage, or both. Anticipated that the foriegn keyed language field will be better.

Risks: Could be redundant with language (from dim tables). Also strings need to be standardized to group similar occurances.

Outstanding questions: none

From dimension table: No

In [74]:
series_name = mdf_non_null.columns[0]
column_eda(series_name)

Null rate: 0.0

accept_language
en-US,en;q=0.9          14015
en-us                    1558
en-US,en;q=0.5            419
en-US,en                  150
en-US,en;q=0.9,es;q=      139
en-US                     100
en-US,en;q=0.9,es-US       37
en-US,en;q=0.9,fr;q=       27
en-GB,en-US;q=0.9,en       25
en-US,en;q=0.9,pt;q=       23
Name: accept_language, dtype: int64


# Carrier

- Cell phone carrier for those accessing from mobile connection
- Should be kept and stored.
- High null rate likely explained by non-mobile connections.

Risks: None

Outstanding questions: none

From dimension table: No


In [75]:
series_name = mdf_non_null.columns[1]
column_eda(series_name)

Null rate: 75.96

carrier
myvzw.com:verizon business                 1574
t-mobile.com:t-mobile usa inc.             1376
att.net:att mobility llc                    490
att.com:att mobility llc                    400
mycingular.net:att mobility llc              93
tmodns.net:t-mobile usa inc.                 29
:mtn nigeria                                 11
d1-online.com:t-mobile deutschland gmbh       9
digicelbroadband.com:digicel jamaica          5
prtc.net:puerto rico telephone company        4
Name: carrier, dtype: int64


# DateTime

- Timestamp of the db entry

Risks: None

Outstanding questions: None

From dimensional table: No

In [76]:
series_name = mdf_non_null.columns[2]
column_eda(series_name)

Null rate: 0.0

date_time
2021-12-16 16:19:58    4
2021-12-16 09:07:18    4
2021-12-16 14:41:49    4
2021-12-16 14:43:51    4
2021-12-16 08:22:38    3
2021-12-16 07:27:51    3
2021-12-16 12:16:25    3
2021-12-16 14:25:59    3
2021-12-16 11:42:28    3
2021-12-16 09:02:29    3
Name: date_time, dtype: int64


# Domain

- ISP of accessor
- Include in dataset

Risks: None

Outstanding questions: None

From dimensional table: No

In [81]:
series_name = mdf_non_null.columns[3]
column_eda(series_name)

Null rate: 0.0

domain
sbcglobal.net             2041
spectrum.com              1636
myvzw.com                 1574
t-mobile.com              1376
comcast.net               1301
knology.net               1142
cloudflare.com             768
googlefiber.net            749
akamaitechnologies.com     689
mchsi.com                  684
Name: domain, dtype: int64


# Exclude hit

- Always 0
- No need to include in dataset at this point

Risks: None

Outstanding questions: Will we use this metric in the future?

From dimensional table: No

In [83]:
series_name = mdf_non_null.columns[4]
column_eda(series_name)

Null rate: 0.0

exclude_hit
0    16786
Name: exclude_hit, dtype: int64


# First hit page URL

- Landing page URL when coming to redfcu.org
- Include in dataset

Risks: None

Outstanding questions: How can this be null? Do we need to fix something on the Adobe side?

From dimensional table: No

In [85]:
series_name = mdf_non_null.columns[5]
column_eda(series_name)

Null rate: 17.42

first_hit_page_url
https://www.redfcu.org/                                       6592
https://www.redfcu.org/signin-page/                           4520
https://www.redfcu.org/personal/banking/online-banking/        800
https://www.redfcu.org/contact-page/                           343
https://redfcu.org/                                            176
https://www.redfcu.org/location/                                89
https://www.redfcu.org/personal/tool-discounts/rfcu-rates/      73
https://www.redfcu.org/personal/banking/mobile-banking/         55
https://www.redfcu.org/rk/                                      55
https://www.redfcu.org/?dsbl_mobi_redir=yes                     47
Name: first_hit_page_url, dtype: int64


# First hit pagename

- Landing page title when coming to redfcu.org
- Include in dataset

Risks: None

Outstanding questions: How can this be null? Do we need to fix something on the Adobe side?

From dimensional table: No

In [87]:
series_name = mdf_non_null.columns[6]
column_eda(series_name)

Null rate: 17.42

first_hit_pagename
redfcu|homepage                                6836
redfcu|signin page                             4549
redfcu|personal|banking|online banking          805
redfcu|contact page                             345
redfcu|personal|loans|auto loans                101
redfcu|personal|loans|mortgages|buy              99
redfcu|location                                  89
redfcu|rk                                        81
redfcu|personal|tool discounts|rfcu rates        73
redfcu|personal|loans|first time home buyer      63
Name: first_hit_pagename, dtype: int64


# First hit referrer

- Search engine that brought user to redfcu.org.
- Include in dataset

Risks: None

Outstanding questions: How can the first hit referrer be redfcu.org? How is redfcu.org search not the first hit? Perhaps adjustments need to be made to adobe for this.

From dimensional table: No

In [88]:
series_name = mdf_non_null.columns[7]
column_eda(series_name)

Null rate: 10.83

first_hit_referrer
https://www.google.com/                                                    10690
https://www.bing.com/                                                       1515
https://www.google.com                                                      1154
https://www.redfcuonline.org/                                                397
https://search.yahoo.com/                                                    274
https://www.redfcu.org/                                                      121
https://duckduckgo.com/                                                      105
android-app://com.google.android.googlequicksearchbox/                        86
http://go.redfcu.org/                                                         69
https://www.redfcuonline.org/tob/live/usp-core/app/logout?reason=logout       45
Name: first_hit_referrer, dtype: int64


# First hit time GMT

- Odd format of time when first hit was made
- Include in dataset

Risks: None

Outstanding questions: How to change this format to GMT/CST? Unclear how to do this.

From dimensional table: No

In [89]:
series_name = mdf_non_null.columns[8]
column_eda(series_name)

Null rate: 0.0

first_hit_time_gmt
1639097761    32
1631809408    19
1632679368    18
1632058487    18
1636819162    18
1616894520    17
1623877908    17
1634489854    16
1639697230    16
1615989195    16
Name: first_hit_time_gmt, dtype: int64


# Geo City

- Physical location city of accessor at time of access.
- Include in dataset

Risks: None

Outstanding questions: None

From dimensional table: No

In [91]:
series_name = mdf_non_null.columns[9]
column_eda(series_name)

Null rate: 0.0

geo_city
huntsville      3507
madison         1404
atlanta         1047
birmingham       722
decatur          676
bellevue         621
athens           539
ashburn          431
nashville        296
murfreesboro     237
Name: geo_city, dtype: int64


# Geo Country

- Physical location country of accessor at time of access.
- Include in dataset

Risks: None

Outstanding questions: Are all the odd locations VPNs? Do we really have this many accesses from Germany? Possibly our members going abroad?

From dimensional table: No

In [92]:
series_name = mdf_non_null.columns[10]
column_eda(series_name)

Null rate: 0.0

geo_country
usa    16626
deu       27
nga       19
ind        8
mex        8
gha        7
abw        6
can        6
tha        6
gbr        6
Name: geo_country, dtype: int64


# Geo DMA

- Unsure of what DMA means. Need to ask someone for know-how on this info.
- Include in dataset

Risks: None

Outstanding questions: Follow up on what dma means.

From dimensional table: No

In [93]:
series_name = mdf_non_null.columns[11]
column_eda(series_name)

Null rate: 0.0

geo_dma
691    8494
524    1180
630    1151
659     929
511     680
819     636
517     266
522     173
602     165
575     158
Name: geo_dma, dtype: int64


# Geo Region

- Physical location state of accessor at time of access.
- Include in dataset

Risks: None

Outstanding questions: None

From dimensional table: No

In [95]:
series_name = mdf_non_null.columns[12]
column_eda(series_name)

Null rate: 0.0

geo_region
al    9731
tn    1396
ga    1305
wa     642
va     579
fl     347
nc     314
il     261
in     246
oh     239
Name: geo_region, dtype: int64


# Geo Zip

- Physical location Zipcode of accessor at time of access.
- Include in dataset

Risks: None

Outstanding questions: Some are null? Unable to tag city, state with some zips?

From dimensional table: No

In [97]:
series_name = mdf_non_null.columns[13]
column_eda(series_name)

Null rate: 0.19

geo_zip
35758    1090
35810     795
35801     745
30319     714
98006     623
35811     550
35803     513
35603     485
20147     433
35242     427
Name: geo_zip, dtype: int64


# Build the final DF for incorta (don't wory about actually building it in python because we'll it in SQL)

what dimensions do we need?
What is the mapping of "included" columns from ADobe name to plain language name in incorta

# Build the excel dictionary:
1. adboe name
2. incorta name
3. descp
4. whatever else you think would be helpful

# Get approval

# Get raw data loaded to incorta

# Write the SQL to make final_df happen for real in SQL using your list and your mapping of column names
# Use excel formla and concentation to do select X as Y automatically.
    = concatenate("t1.",A1," as ",B1,",")


In [ ]:
mobile_df = MDF[MDF['mobile']==1]
carrier_df = mobile_df.groupyb(carrier).agg(coint).reste_index().sort_values(xcvxc)
display(carrier_df.head(10))

In [3]:
null_cols_check = ['post_campaign', 'post_evar2', 'post_evar6', 'post_evar7', 'post_evar8', 'post_evar11', 'post_evar12', 'post_evar13', 'post_evar14',\
                   'post_evar16', 'post_evar17', 'post_evar18', 'post_evar19', 'post_evar21', 'post_evar24', 'post_evar25', 'post_evar26', 'post_evar27',\
                   'post_evar28', 'post_evar29', 'post_evar30', 'post_evar31', 'post_evar32', 'post_evar33', 'post_evar34', 'post_evar35', 'post_evar36',\
                   'post_evar37', 'post_evar38', 'post_evar39', 'post_evar40', 'post_evar41', 'post_evar42', 'post_evar43', 'post_evar44', 'post_evar45',\
                   'post_evar46', 'post_evar47', 'post_evar48', 'post_evar49', 'post_evar50', 'post_evar51', 'post_evar52', 'post_evar53', 'post_evar54',\
                   'post_evar55', 'post_evar56', 'post_evar57', 'post_evar58', 'post_evar59', 'post_evar60', 'post_evar61', 'post_evar62', 'post_evar63',\
                   'post_evar64', 'post_evar65', 'post_evar66', 'post_evar67', 'post_evar68', 'post_evar69', 'post_evar70', 'post_evar71', 'post_evar72',\
                   'post_evar73', 'post_evar74', 'post_evar75', 'post_evar76', 'post_evar77', 'post_evar78', 'post_evar79', 'post_evar80', 'post_evar81',\
                   'post_evar82', 'post_evar83', 'post_evar84', 'post_evar85', 'post_evar86', 'post_evar87', 'post_evar88', 'post_evar89', 'post_evar90',\
                   'post_evar91', 'post_evar92', 'post_evar93', 'post_evar94', 'post_evar95', 'post_evar96', 'post_evar97', 'post_evar98', 'post_evar99',\
                   'post_evar100', 'post_hier1', 'post_hier2', 'post_hier3', 'post_hier4', 'post_hier5', 'post_page_event_var3', 'post_page_type', 'post_prop2',\
                   'post_prop3', 'post_prop4', 'post_prop5', 'post_prop6', 'post_prop7', 'post_prop8', 'post_prop11', 'post_prop12', 'post_prop13', 'post_prop14',\
                   'post_prop15', 'post_prop16', 'post_prop18', 'post_prop19', 'post_prop20', 'post_prop21', 'post_prop22', 'post_prop23', 'post_prop24', 'post_prop25',\
                   'post_prop28', 'post_prop29', 'post_prop30', 'post_prop31', 'post_prop32', 'post_prop33', 'post_prop34', 'post_prop35', 'post_prop36', 'post_prop37',\
                   'post_prop38', 'post_prop39', 'post_prop40', 'post_prop41', 'post_prop42', 'post_prop43', 'post_prop44', 'post_prop45', 'post_prop46', 'post_prop47',\
                   'post_prop48', 'post_prop49', 'post_prop50', 'post_prop51', 'post_prop52', 'post_prop53', 'post_prop54', 'post_prop55', 'post_prop56', 'post_prop57',\
                   'post_prop58', 'post_prop59', 'post_prop60', 'post_prop61', 'post_prop62', 'post_prop63', 'post_prop64', 'post_prop65', 'post_prop66', 'post_prop67',\
                   'post_prop68', 'post_prop69', 'post_prop70', 'post_prop71', 'post_prop72', 'post_prop73', 'post_prop74', 'post_prop75', 'post_purchaseid', 'post_state']


In [4]:
# Checking for columns with only null values in them
null_cols = []
for column in mdf:
    all_null = pd.isnull(mdf[column]).all()
    if all_null == True:
        null_cols.append(column)

# Check the .tsv read in on this one with the control, to make sure that the same columns are 100% null values
if null_cols == null_cols_check:
    print('No new null columns, continue')
else:
    print("There are new columns with data populated into them. Update null_cols_check and make sure not to drop columns with values in them")

No new null columns


In [5]:
# Drop the null columns from the dataframe.
mdf = mdf.drop(labels = null_cols, axis = 1)
#print(mdf.info(verbose = True))
mdf.head(20).to_csv('Filtered_example.csv')
col_names = mdf.columns.values.tolist()

In [9]:
# Here we are going to get a non-null example for each column
examples = []
for column in mdf:
    index = mdf[column].first_valid_index()
    first_val = mdf.loc[index,column]
    examples.append(first_val)
print(examples)
examples = pd.Series(examples)
examples.to_csv('examples_non_null_vals.csv')

['en-US,en;q=0.9', 3151744350, 't-mobile.com:t-mobile usa inc.', 2, 300, '2021-12-16 01:48:22', 'microsoft.com', 0, 'https://www.redfcu.org/', 'redfcu|homepage', 'https://www.google.com/', 1639640902, 'redmond', 'usa', 819, 'wa', '98052', '131.107.1.226', 7, 45, 0, 0, 0, '76717311758226592931862399968198873945', 0, 1, 1240087047, 905, 1280, 'signin page', 'Y', 'USD', 1639640902, 'https://www.redfcu.org/', 'redfcu|homepage', 'sign in', 10.0, 'New', 'First Visit', '$200 off first full month’s rent*', 'current location', 11.0, '76717311758226592931862399968198873945', '100,102,108,109,122', '::empty::', 0, 'tel:8002341234', 'click to call', 'https://www.redfcu.org', 'redfcu|homepage', 'redfcu|homepage', 'Y', ';;;;', 'https://www.redfcu.org/', 'New', 'First Visit', '2:48 AM|Thursday', 'highestPercentViewed=11', 'redfcu|homepage', 'https://www.google.com/', 0, 47988631485083160, 6084104561032483093, '::hash::0', 0, 'google.com', 6, 13, 'ss', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleW

In [8]:
# Making a list of all unique values for each column
uniques_df = pd.Series({col:mdf[col].unique() for col in mdf})
print(uniques_df)
uniques_df.to_csv('unique_values_per_column.csv')

accept_language           [en-US,en;q=0.9, en-us, en-US,en-GB;q=0.9,en, ...
browser                   [3151744350, 2936625532, 3848701173, 257280662...
carrier                   [nan, t-mobile.com:t-mobile usa inc., att.net:...
connection_type                                                      [2, 4]
country                   [300, 304, 0, 302, 305, 232, 303, 84, 117, 71,...
date_time                 [2021-12-16 01:48:22, 2021-12-16 01:49:43, 202...
domain                    [microsoft.com, wlv.com, amazonaws.com, t-mobi...
exclude_hit                                                             [0]
first_hit_page_url        [https://www.redfcu.org/, https://redfcu.org/,...
first_hit_pagename        [redfcu|homepage, redfcu|personal|credits card...
first_hit_referrer        [nan, https://www.google.com/, https://www.red...
first_hit_time_gmt        [1639640902, 1615895708, 1639641173, 163963727...
geo_city                  [redmond, decatur, dublin, birmingham, apopka,...
geo_country 